<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 6: Использование Advantage Actor Critic (A2C) для моделирования робототехники с помощью PyBullet и Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail" width="50%"/>

В этом блокноте вы научитесь использовать A2C с двумя наборами робототехнических сред PyBullet и Panda-Gym.

С [PyBullet](https://github.com/bulletphysics/bullet3), мы собираемся **обучить робота двигаться**:
- `AntBulletEnv-v0` 🕸️ Точнее, паук (они говорят Муравей, но да ладно... это паук 😆 ) 🕸️

Затем, с [Panda-Gym](https://github.com/qgallouedec/panda-gym), мы собираемся **обучить роботизированную руку** (Франка Эмика Панда робот) выполнять задание:
- `Reach`: робот должен поместить свой конечный манипулятор в целевое положение.

После этого вы сможете **тренироваться в других средах робототехники**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/environments.gif" alt="Robotics environments" width="50%"/>

### 🎮 Окружающая среда: 

- [PyBullet](https://github.com/bulletphysics/bullet3)
- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 RL-Библиотека: 

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

## Цели этого блокнота 🏆

В конце этого блокнота вы:

- Научитесь использовать библиотеки сред **PyBullet** и **Panda-Gym**.
-  Научитесь **обучать роботов, используя A2C**.
- Поймете, почему **нам нужно нормализовать входные данные**.
- Будите в состоянии **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥.

## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

🔲 📚 [Метод "актер-критик" Блок 6](https://huggingface.co/deep-rl-course/unit6/introduction) 🤗  

# Давайте обучим наших первых роботов 🤖

Чтобы подтвердить это практическое руководство для [процесса сертификации](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process),  вам нужно отправить две ваши обученные модели в концентратор и получить следующие результаты:

- `AntBulletEnv-v0` получив результат >= 650.
- `PandaReachDense-v2` получив результат >= -3.5.

Чтобы найти свой результат, перейдите к [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) и найдите свою модель, **the result = mean_reward - std of reward**

Если вы не нашли свою модель, **перейдите в нижнюю часть страницы и нажмите на кнопку обновить**

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Создайте виртуальный дисплей 🔽

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab **нам нужен виртуальный экран, чтобы иметь возможность отображать среду** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Установка зависимостей 🔽
Первым шагом является установка библиотек, мы установим несколько из них:

- `pybullet`: Содержит среду обитания шагающих роботов.
- `panda-gym`: Содержит среду роботизированной руки.
- `stable-baselines3[extra]`: Библиотека обучения с глубоким подкреплением SB3.
- `hugging face_sb3`: Дополнительный код для Stable-baselines 3 для загрузки моделей из Hugging Face 🤗 Hub.
- `hugging face_hub`: Библиотека, позволяющая любому пользователю работать с репозиториями хаба.

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit6/requirements-unit6.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.3 MB/s eta 0:00:00
  Us

## Импортируйте пакеты 📦

In [4]:
import pybullet_envs
import panda_gym
import gym

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

## Окружающая среда №1: AntBulletEnv-v0 🕸


### Создайте AntBulletEnv-v0
#### Окружающая среда 🎮
В этой среде агент должен правильно использовать свои различные суставы, чтобы правильно ходить.
Вы можете найти подробное объяснение этой среды здесь: https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet

In [5]:
env_id = "AntBulletEnv-v0"
# Создайте окружающую среду
env = gym.make(env_id)

# Получите пространство состояний и пространство действий
s_size = env.observation_space.shape[0]
a_size = env.action_space

In [6]:
print("_____ПРОСТРАНСТВО НАБЛЮДЕНИЙ_____ \n")
print("Размер пространства состояний: ", s_size)
print("Выборочное наблюдение", env.observation_space.sample()) # Получите случайное наблюдение

_____ПРОСТРАНСТВО НАБЛЮДЕНИЙ_____ 

Размер пространства состояний:  28
Выборочное наблюдение [-0.05873336  2.395351   -0.40879452  0.939623    0.9174361  -0.291952
 -0.9814798   0.27011833  0.82616293 -0.30532253 -0.4916162  -0.7449047
 -0.30217567 -0.49502516  1.0123965   0.970613    0.2857124  -0.97599745
 -0.41402417  1.2762734  -1.7961597   0.11626722  0.60692835  0.06346635
 -1.0124248  -0.18815549  0.23635985 -0.7974657 ]


Пространство наблюдения (от [Jeffrey Y Mo](https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet)):

Разница в том, что наше пространство наблюдения равно 28, а не 29.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/obs_space.png" alt="PyBullet Ant Obs space" width="70%"/>

In [7]:
print("\n _____ПРОСТРАНСТВО ДЕЙСТВИЙ_____ \n")
print("Размер пространства действий: ", a_size)
print("Пример действия из пространства", env.action_space.sample()) # Получите случайное действие


 _____ПРОСТРАНСТВО ДЕЙСТВИЙ_____ 

Размер пространства действий:  Box([-1. -1. -1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1. 1. 1.], (8,), float32)
Пример действия из пространства [-0.6806766  -0.71949154 -0.7696165   0.78447986 -0.85347444  0.9047551
 -0.3724377   0.32215872]


Пространство действий (от [Jeffrey Y Mo](https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet)):

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/action_space.png" alt="PyBullet Ant Obs space" width="70%"/>

### Нормализуйте наблюдение и вознаграждения

Хорошей практикой в обучении с подкреплением является [нормализация входных характеристик](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html). 

Для этой цели существует оболочка, которая вычислит текущее среднее значение и стандартное отклонение входных признаков.

Мы также нормализуем вознаграждения с помощью этой же оболочки, добавляя `norm_reward = True`

[Вам следует ознакомиться с документацией, чтобы заполнить эту ячейку](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize)

In [ ]:
env = make_vec_env(env_id, n_envs=4)

# Добавление этой оболочки для нормализации наблюдения и вознаграждения
env = # ЗАДАЧА: Добавить оболочку

#### Решение

In [8]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

### Создание модели A2C 🤖

В этом случае, поскольку у нас есть вектор из 28 значений в качестве входных данных, мы будем использовать MLP (многослойный персептрон) в качестве политики.

Для получения дополнительной информации о реализации A2C со  StableBaselines3 проверьте: https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html#notes

Чтобы найти наилучшие параметры, я проверил [официальные обученные агенты командой Stable-Baselines 3](https://huggingface.co/sb3).

In [ ]:
model = # Создайте модель A2C и постарайтесь найти наилучшие параметры

#### Решение

In [9]:
model = A2C(policy = "MlpPolicy",
            env = env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            use_sde= True,
            verbose=1)

Using cuda device


### Обучите агента A2C 🏃
- Давайте обучим нашего агента на 2 000 000 временных шагов, не забудьте использовать GPU на Colab. Это займет примерно ~25-40 минут

In [10]:
model.learn(2_000_000)

Выходные данные были обрезаны до нескольких последних строк (5000).
| time/                 |          |
|    fps                | 575      |
|    iterations         | 34800    |
|    time_elapsed       | 1934     |
|    total_timesteps    | 1113600  |
| train/                |          |
|    entropy_loss       | -3.3     |
|    explained_variance | 0.983    |
|    learning_rate      | 0.00096  |
|    n_updates          | 34799    |
|    policy_loss        | 0.0613   |
|    std                | 0.0608   |
|    value_loss         | 0.00132  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1e+03    |
|    ep_rew_mean        | 1.49e+03 |
| time/                 |          |
|    fps                | 575      |
|    iterations         | 34900    |
|    time_elapsed       | 1939     |
|    total_timesteps    | 1116800  |
| train/                |          |
|    entropy_loss       | -3.15    |
|    ex

In [11]:
# Сохраните модель, и Vec Normalize  статистику при сохранении агента
model.save("a2c-AntBulletEnv-v0")
env.save("vec_normalize.pkl")

### Оцените агента 📈
- Теперь, когда наш агент обучен, нам нужно **проверить его работоспособность**.
- Stable-Baselines 3 предоставляет метод для этого: `evaluate_policy`
- В моем случае я получил скромную награду в размере `2371.90 +/- 16.50 `

In [12]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("AntBulletEnv-v0")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-AntBulletEnv-v0")

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = 1970.15 +/- 62.26


### Опубликуйте свою обученную модель на хабе 🔥
Теперь, когда мы увидели, что получили хорошие результаты после обучения, мы можем опубликовать нашу обученную модель на хабе с помощью одной строки кода.

📚 Документация библиотек 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20

Вот пример карты модели (со встроенной средой):

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/modelcardpybullet.png" alt="Model Card Pybullet" width="90%"/>

Используя `package_to_hub`, как мы уже упоминали в предыдущих разделах, **вы оцениваете, записываете повтор, генерируете карточку модели вашего агента и отправляете ее в хаб**.

Таким образом:
- Вы можете **продемонстрировать свою работу** 🔥
- Вы можете **визуализировать, как ваш агент играет** 👀
- Вы можете **поделиться с сообществом агентом, которым могут пользоваться другие** 💾
- Вы можете **получить доступ к таблице лидеров 🏆 , чтобы увидеть, насколько хорошо работает ваш агент по сравнению с вашими одноклассниками** 👉 [Leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard).

In [13]:
notebook_login()
!git config --global credential.helper store

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Art-phys/a2c-{env_id}", # Измените имя пользователя
    commit_message="Initial commit",
)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpeijialrr/-step-0-to-step-1000.mp4
ℹ Pushing repo Art-phys/a2c-AntBulletEnv-v0 to the Hugging Face Hub


policy.optimizer.pth:   0%|          | 0.00/56.2k [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/57.0k [00:00<?, ?B/s]

...

replay.mp4:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

a2c-AntBulletEnv-v0.zip:   0%|          | 0.00/129k [00:00<?, ?B/s]

vec_normalize.pkl:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Art-phys/a2c-AntBulletEnv-v0/tree/main/


'https://huggingface.co/Art-phys/a2c-AntBulletEnv-v0/tree/main/'

## Окружающая среда 2: Panda Reach Dense-v2 🦾

Агент, которого мы собираемся обучить, - это роботизированная рука, которая должна выполнять управление (двигать рукой и использовать конечный эффектор).

В робототехнике конечный эффектор - это устройство на конце роботизированной руки, предназначенное для взаимодействия с окружающей средой.

В режиме "Panda Reach" робот должен поместить свой конечный эффектор в целевое положение (зеленый шар).

Мы собираемся использовать плотную версию этой среды. Это означает, что мы получим *плотную функцию вознаграждения*, которая **будет предоставлять вознаграждение на каждом временном шаге** (чем ближе агент к выполнению задачи, тем выше вознаграждение). В отличие от *разреженной функции вознаграждения*, где среда **возвращает вознаграждение тогда и только тогда, когда задача выполнена**.

Кроме того, мы собираемся использовать *Управление смещением конечного эффектора*, это означает, что действие **соответствует смещению конечного эффектора**. Мы не контролируем индивидуальное движение каждого сустава (joint control).

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/robotics.jpg"  alt="Robotics" width="70%"/>


Таким образом **тренировка будет проще**.


В `PandaReachDense-v2` роботизированная рука должна поместить свой конечный эффектор в целевое положение (зеленый шар).

In [21]:
import gym

env_id = "PandaReachDense-v2"

# Создайте среду
env = gym.make(env_id)

# Получите пространство состояний и пространство действий
s_size = env.observation_space.shape
a_size = env.action_space

In [22]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Получите случайное наблюдение

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([-7.7541533,  9.682014 , -2.3775086], dtype=float32)), ('desired_goal', array([0.03242053, 4.005729  , 0.94964516], dtype=float32)), ('observation', array([-5.3415203 , -8.817021  , -9.765576  ,  6.2947755 , -0.61150056,
       -7.0374155 ], dtype=float32))])


Пространство наблюдения **представляет собой словарь с 3 различными элементами**:
- `achieved_goal`: (x,y,z) положение цели.
- `desired_goal`: (x,y,z) расстояние между положением цели и текущим положением объекта.
- `observation`: положение (x, y,z) и скорость конечного эффектора (vx, vy, vz).

Учитывая, что это словарь в качестве наблюдения, **нам нужно будет использовать политику множественного ввода MultiInputPolicy вместо MlpPolicy**.

In [23]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Получите случайное действие


 _____ACTION SPACE_____ 

The Action Space is:  Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)
Action Space Sample [ 0.00626361 -0.73918056  0.6125092 ]


Пространство действий представляет собой вектор с 3 значениями:
- Управляйте движением x, y, z

Теперь твоя очередь:

1. Определите среду под названием "PandaReachDense-v2".
2. Создайте векторизованную среду
3. Добавьте оболочку для нормализации наблюдений и вознаграждений. [Проверьте документацию](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize )
4. Создайте модель A2C (не забудьте verbose=1, чтобы распечатать журналы обучения).
5. Тренируйте его в течение 1 м временных шагов
6. Сохраните модель и VecNormalize статистику при сохранении агента
7. Оцените своего агента
8. Опубликуйте свою обученную модель на хабе 🔥 с помощью `package_to_hub`

### Решение (заполните задачу)

In [ ]:
# 1 - 2
env_id = "PandaReachDense-v2"
env = make_vec_env(env_id, n_envs=4)

# 3
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

# 4
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)
# 5
model.learn(1_000_000)

Using cuda device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50       |
|    ep_rew_mean        | -12.7    |
| time/                 |          |
|    fps                | 216      |
|    iterations         | 100      |
|    time_elapsed       | 9        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.29    |
|    explained_variance | 0.503    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -3.28    |
|    std                | 1.01     |
|    value_loss         | 1.38     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 50       |
|    ep_rew_mean        | -13.6    |
| time/                 |          |
|    fps                | 285      |
|    iterations         | 200      |
|    time_elapsed       | 14       |
|    total_timesteps

In [ ]:
# 6
model_name = "a2c-PandaReachDense-v2"; 
model.save(model_name)
env.save("vec_normalize.pkl")

# 7
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Загрузите сохраненную статистику
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v2")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  не обновляйте их во время тестирования
eval_env.training = False
# нормализация вознаграждения во время тестирования не требуется
eval_env.norm_reward = False

# Загрузите агент
model = A2C.load(model_name)

mean_reward, std_reward = evaluate_policy(model, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

# 8
package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Art-phys/a2c-{env_id}", # ЗАДАЧА: Измените имя пользователя
    commit_message="Initial commit",
)

## Некоторые дополнения 🏆
Лучший способ научиться **- это пробовать что-то самостоятельно**! Почему бы не попробовать `HalfCheetahBulletEnv-v0` для PyBullet и `PandaPickAndPlace-v1` для Panda-Gym?

Если вы хотите попробовать более сложные задачи для panda-gym, вам нужно проверить, что было сделано с помощью **QC или SAC** (более эффективный алгоритм, подходящий для задач робототехники). В реальной робототехнике вы будете использовать более эффективный алгоритм выборки по простой причине: в отличие от симуляции **если вы слишком сильно двигаете роботизированной рукой, у вас есть риск сломать ее**.

PandaPickAndPlace-v1: https://huggingface.co/sb3/tqc-PandaPickAndPlace-v1

И не стесняйтесь ознакомиться с документацией panda-gym здесь: https://panda-gym.readthedocs.io/en/latest/usage/train_with_sb3.html

Вот несколько идей для достижения этого:
* Тренируйте больше шагов
* Попробуйте разные гиперпараметры, посмотрев на то, что сделали ваши одноклассники 👉 https://huggingface.co/models?other=https://huggingface.co/models?other=AntBulletEnv-v0
* **Разместите свою новую обученную модель** на хабе 🔥

Увидимся в блоке 7! 🔥
## Keep learning, stay awesome 🤗